This project aims at simulating on your computer the paramagnetic/ferromagnetic phase transition. For this, we will simulate the 2 dimensional ising model.

# Ising model

The hamiltonian (without external magnetic field) is

$$H = -J\sum_{(i,j)}S_i S_j$$

where $S_i$ is the spin at location $i$ and $J$ is a constant describing the strength of the interaction between neighbouring spins. We consider $S_i = \pm 1$ (for all $i$). The sum in the previous equation runs overs all neaghbouring spins. In order to simulate the system, we will implement the Metropolis algorithm, which proceeds as follows:

1. Start with some random spin configuration
2. Pick a spin at random and flip it
3. Compute the energy change $\Delta E$ due to this flip
4. If $\Delta E \leq 0$, accept the flip
5. If $\Delta E > 0$, accept the flip with probability $P = e^{-\beta \Delta E}$, where $\beta = \frac{1}{k_BT}$
6. If flip is rejected, put the spin back in its original direction
7. Go to 2. until maximum number of iteractions is reached

# Implementation

In order to simplify the implementation, we will take $J=1$ and $k_B=1$. 

## Step 1

The first step of the implementation consists in creating a 2D grid of spins in up and down positions and plotting it. For this, do the following:

- Implement a `struct` called `Grid` having as fields: a 2D slice (that will contain the spin values at each node of the grid), the size $N$ of the grid ($N$ corresponds to the number of rows and columns, so that the total number of spins is $N\times N$), the coupling $J$ and the temperature $T$.
- Implement a `NewGrid` function that builds an object of type `Grid` (take $N=80$). 
- Implement a method called `Init` that randomly initializes the grid.

In the last step, one need to draw random numbers. For this, the `math/rand` package can be used. Once this is done, a plot displaying the grid must be produced. For this, we will use the `gonum/plot` package. The idea is to create two scatter plots (one containing the coordinates of the up spins and the other one containing the coordinates of the down spins) and to plot them on top of each other. Scatter plots can be created using the `plotter.Scatter` structure. This structure requires a `struct` implementing the `XYer` interface (defined in plotter/plotter.go). You thus need to implement such a `struct` and create two instances of it, one for the up spins and one for the down spins. 

Now that you have your two `plotter.Scatter` objects `scaUp` and `scaDown` you can plot them. For this, do:

```
p, _ := plot.New()
p.Add(scaUp, scaDown)
p.Save(6*vg.Inch, 6*vg.Inch, "Grid2D.png")
```

The first line creates a plot object. The second line adds to the plot your two scatter plots and the third line saves your plot into a png file. Open this png file and check that it makes sense. 

## Remaining steps


